In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [6]:
df = pd.read_csv("Online Retail.csv")

In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [8]:
df_pivot = df.pivot_table(index=['CustomerID'],
                columns=['StockCode'],
                values='Quantity',
                fill_value=0).reset_index()

In [9]:
purchase_counts = df_pivot.groupby('CustomerID').sum()

In [10]:
purchase_counts

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0.0,0,0.0,0,0.0,0,0.00
12347.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0.0,0,0.0,0,0.0,0,0.00
12348.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0.0,0,0.0,0,0.0,0,2.25
12349.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0.0,0,0.0,0,0.0,0,1.00
12350.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0.0,0,0.0,0,0.0,0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0.0,0,0.0,0,0.0,0,0.00
18281.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0.0,0,0.0,0,0.0,0,0.00
18282.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0.0,0,0.0,0,0.0,0,0.00


In [11]:
customer_similarities = cosine_similarity(purchase_counts)

In [22]:
customer_similarities[1].max()

1.0000000000000002

In [20]:
len(df['CustomerID'].unique())

4373